In [1]:
#Importación de librerias
from imutils.video import VideoStream
from imutils.video import FPS
import imutils
import numpy as np
import cv2
import time

In [211]:
def auto_canny(image, sigma=0.33):
	# compute the median of the single channel pixel intensities
	v = np.median(image)

	# apply automatic Canny edge detection using the computed median
	lower = int(max(0, (1.0 - sigma) * v))
	upper = int(min(255, (1.0 + sigma) * v))
	edged = cv2.Canny(image, lower, upper)

	# return the edged image
	return edged

color_dict_HSV = {'black': [[180, 255, 30], [0, 0, 0]],
              'white': [[180, 18, 255], [0, 0, 231]],
              'red1': [[180, 255, 255], [159, 50, 70]],
              'red2': [[9, 255, 255], [0, 50, 70]],
              'green': [[89, 255, 255], [36, 50, 70]],
              'blue': [[128, 255, 255], [90, 50, 70]],
              'yellow': [[35, 255, 255], [25, 50, 70]],
              'purple': [[158, 255, 255], [129, 50, 70]],
              'orange': [[24, 255, 255], [10, 50, 70]],
              'gray': [[180, 18, 230], [0, 0, 40]],
			  'brown': [[25, 100, 97], [24, 33, 18]]}


In [192]:
# Buscar Colores
gray = np.uint8([[[36, 52, 79]]])
hsv_gray = cv2.cvtColor(gray,cv2.COLOR_BGR2HSV)
print( hsv_gray )

[[[ 11 139  79]]]


In [215]:
# Rangos para filtros HSV
lower_color = np.array([5, 70 , 60])
upper_color = np.array([50, 150, 255])

# Inicializar video streams
print("[INFO] accessing video stream...")
cap = cv2.VideoCapture("IMG_3077.mov")

#Verificar si la camara esta abiera
if (cap.isOpened()==False):
	print("[INFO] No fue posible abrir el video, favor reintente")

#Leer el video hasta que se complete
while(cap.isOpened()):
	#Leer frame-by-frame
	ret, frame = cap.read()
	if ret == True:
		frame = imutils.resize(frame, height=600)
		
		#-----------------------------------------------------------------------
		# Filtro de color
		hsv_img = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
		# Configurar mascara
		mask = cv2.inRange(hsv_img, lower_color, upper_color)
		# Aplicar mascara
		res = cv2.bitwise_and(frame, frame, mask=mask)
		# Resultados
		cv2.imshow('mask', mask)
		cv2.imshow('Filtered Color Only', res)

		#-----------------------------------------------------------------------
		# Video en grayscale
		roi = res[300:500, 0:300]
		gray = cv2.cvtColor(roi, cv2.COLOR_BGR2GRAY)
		blurred = cv2.GaussianBlur(gray, (5, 5), 0)
		thresh = cv2.threshold(blurred, 125, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)[1]
		#Aplicar Auto Canny
		edges = auto_canny(thresh)

		#------------------------------------------------------------------------
		##Usar componentes conectados para filtrar datos

		#AQUI...


		# Encontrar contornos
		contours, _ = cv2.findContours(gray, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
		
		# Por cada contorno...
		for cnt in contours:
			(x, y, w, h) = cv2.boundingRect(cnt)
			# Calcula el area
			area = cv2.contourArea(cnt)
			
			# Distinguir entre caja grande, mediana o pequeña
			if area > 100 and area <= 500:
				# Caja pequeña
				cv2.rectangle(roi, (x, y), (x + w, y + h), (0, 0, 255), 2)
				cv2.putText(roi, "Caja pequeña - Area: " + str(area), (x,y+30), 1, 1, (0,0,255))
			elif area > 500 and area <= 900:
				# Caja mediana
				cv2.rectangle(roi, (x, y), (x + w, y + h), (255, 0, 0), 2)
				cv2.putText(roi, "Caja mediana - Area: " + str(area), (x,y+30), 1, 1, (255,0,0))
			elif area > 900:
				# Caja grande
				cv2.rectangle(roi, (x, y), (x + w, y + h), (0, 255, 0), 2)
				cv2.putText(roi, "Caja grande - Area: " + str(area), (x,y+30), 1, 1, (0, 255, 0))
			
			#cv2.putText(roi, str(area), (x, y), 1, 1, (0, 255, 0))	
		
		cv2.imshow("Original", frame)
		cv2.imshow("Gray Color", gray)
		cv2.imshow("Canny", edges)
		cv2.imshow("Thresh", thresh)
		cv2.imshow("ROI", roi)

		# Mostrar video
		#cv2.imshow("Original", frame)
		# Si se apreta la tecla q, el video se detiene antes
		if cv2.waitKey(10) & 0xFF == ord("q"):
			break
	else:
		break

#Limpiar cache del video en variable cap
cap.release()
#Cerrar ventanas
cv2.destroyAllWindows()

[INFO] accessing video stream...
